In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/frenchenglish/_about.txt
/kaggle/input/frenchenglish/fra.txt


In [2]:
english_texts=[]
french_texts=[]
english_character=[]
french_character=[]
with open("/kaggle/input/frenchenglish/fra.txt", "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[:30000]:
    english_text,french_text,_=line.split("\t")
    english_texts.append(english_text)
    french_text = "\t" + french_text + "\n"
    french_texts.append(french_text)

In [3]:
for i in english_texts:
    for c in i:
        if c not in english_character:
            english_character.append(c)
            english_character.sort()

In [4]:
for j in french_texts:
    for c in j:
        if c not in french_character:
            french_character.append(c)
            french_character.sort()

In [5]:
english_d={}
for i in range(len(english_character)):
    english_d[english_character[i]]=i

In [6]:
french_d={}
for i in range(len(french_character)):
    french_d[french_character[i]]=i

In [7]:
english_encoder_tokens = len(english_character)
french_decoder_tokens = len(french_character)

In [8]:
max_encoder_seq_length=0
for i in english_texts:
    if len(i)>max_encoder_seq_length:
        max_encoder_seq_length=len(i)
    

In [9]:
max_decoder_seq_length=0
for i in french_texts:
    if len(i)>max_decoder_seq_length:
        max_decoder_seq_length=len(i)

In [10]:
english_texts[0]

'Go.'

In [11]:
encoder_input_data=[]
for bb in range(30000):
    a=[]
    b=[]
    c=[]
    k=len(english_texts[bb])
    m=0
    while m<k:
        for char in english_texts[bb][m]:
                           for i in range(len(english_character)):
                                            if english_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    while m<max_encoder_seq_length:
        for i in range(len(english_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    encoder_input_data.append(c)

In [12]:
encoder_input_data=np.array(encoder_input_data)

In [13]:
encoder_input_data.shape

(30000, 20, 75)

In [14]:
decoder_input_data=[]
for bb in range(30000):
    a=[]
    b=[]
    c=[]
    k=len(french_texts[bb])
    m=0
    while m<k:
        for char in french_texts[bb][m]:
                           for i in range(len(french_character)):
                                            if french_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    while m<max_decoder_seq_length:
        for i in range(len(french_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    decoder_input_data.append(c)

In [15]:
decoder_input_data=np.array(decoder_input_data)

In [16]:
max_decoder_seq_length

59

In [17]:
len(french_character)

100

In [18]:
decoder_input_data.shape

(30000, 59, 100)

In [19]:
decoder_target_data=[]
for bb in range(30000):
    a=[]
    b=[]
    c=[]
    k=len(french_texts[bb])
    m=1
    while m<k:
        for char in french_texts[bb][m]:
                           for i in range(len(french_character)):
                                            if french_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    m=m-1
    while m<max_decoder_seq_length:
        for i in range(len(french_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    decoder_target_data.append(c)

In [20]:
decoder_target_data=np.array(decoder_target_data)

In [21]:
decoder_target_data.shape

(30000, 59, 100)

In [22]:
batch_size = 64  # Batch size for training.
epochs = 100 # Number of epochs to train for.
latent_dim = 256 

In [23]:
from keras.layers import Dense
from keras.layers import LSTM,Input
from keras.models import Model

In [24]:
encoder_inputs = Input(shape=(None,len(english_character)))
encoder = LSTM(latent_dim,dropout=0.2,return_sequences=True,return_state=True)
encoder_outputs_1, state_h_1, state_c_1 = encoder(encoder_inputs)
encoder = LSTM(latent_dim,dropout=0.2,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_outputs_1)
encoder_states = [state_h_1,state_c_1,state_h, state_c]

In [25]:
decoder_inputs = Input(shape=(None, len(french_character)))
decoder_lstm = LSTM(latent_dim,return_sequences=True,dropout=0.2,return_state=True)
decoder_outputs_1, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h_1,state_c_1])
decoder_lstm_1 = LSTM(latent_dim, return_sequences=True,dropout=0.2,return_state=True)
decoder_outputs, _, _ = decoder_lstm_1(decoder_outputs_1, initial_state=[state_h,state_c])
decoder_dense = Dense(len(french_character), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)
model=Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [26]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 75)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 100)]  0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, None, 256),  339968      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  365568      input_2[0][0]                    
                                                                 lstm[0][1]            

In [27]:
model.layers[2]

In [28]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100
375/375 [==============================] - 12s 33ms/step - loss: 1.0128 - accuracy: 0.7234 - val_loss: 0.8561 - val_accuracy: 0.7549
Epoch 2/100
375/375 [==============================] - 10s 27ms/step - loss: 0.6964 - accuracy: 0.7970 - val_loss: 0.7248 - val_accuracy: 0.7862
Epoch 3/100
375/375 [==============================] - 10s 27ms/step - loss: 0.6084 - accuracy: 0.8200 - val_loss: 0.6327 - val_accuracy: 0.8115
Epoch 4/100
375/375 [==============================] - 10s 26ms/step - loss: 0.5608 - accuracy: 0.8329 - val_loss: 0.5852 - val_accuracy: 0.8231
Epoch 5/100
375/375 [==============================] - 10s 27ms/step - loss: 0.5248 - accuracy: 0.8428 - val_loss: 0.5460 - val_accuracy: 0.8364
Epoch 6/100
375/375 [==============================] - 10s 27ms/step - loss: 0.4980 - accuracy: 0.8507 - val_loss: 0.5200 - val_accuracy: 0.8431
Epoch 7/100
375/375 [==============================] - 10s 26ms/step - loss: 0.4778 - accuracy: 0.8566 - val_loss: 0.5022 - val_ac

In [29]:
model.save("engtofrench.h5")

In [30]:
model.layers[3]

In [31]:
model.layers[1]

In [32]:
model.layers[6]

In [33]:
model = keras.models.load_model("engtofrench.h5")
encoder_inputs = model.input[0]  # input_1
encoder_outputs_1, state_h_enc_1, state_c_enc_1 = model.layers[2].output 
encoder_outputs, state_h_enc, state_c_enc = model.layers[4].output 
encoder_states = [state_h_enc_1, state_c_enc_1,state_h_enc, state_c_enc]
encoder_model_1 = keras.Model(encoder_inputs, encoder_states)

In [34]:
decoder_inputs = model.input[1]  
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_state_input_h1 = Input(shape=(latent_dim,),name="input_5")
decoder_state_input_c1 = Input(shape=(latent_dim,),name="input_6")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c,decoder_state_input_h1,decoder_state_input_c1]
decoder_lstm = model.layers[3]
dec_o, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs[:2])
decoder_lstm_1=model.layers[5]
dec_o_1, state_h1, state_c1 = decoder_lstm_1(
    dec_o, initial_state=decoder_states_inputs[-2:])
decoder_states = [state_h,state_c,state_h1,state_c1]
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(dec_o_1)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)



In [35]:
reverse_input_char_index ={}
for i in range(len(english_character)):
    reverse_input_char_index[i]=english_character[i]
reverse_target_char_index ={}
for i in range(len(french_character)):
    reverse_target_char_index[i]=french_character[i]

In [36]:
def decode_sequence(input_seq):
    states_value=encoder_model_1.predict(input_seq)
    target_seq = np.zeros((1, 1, len(french_character)))
    target_seq[0, 0, french_d["\t"]] = 1.0
    flag=0
    sent=""
    while flag==0:
        output_tokens, h, c,h1,c1 = decoder_model.predict([target_seq] + states_value)
        sample = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sample]
        sent+=sampled_char
        if sampled_char == "\n" or len(sent) > max_decoder_seq_length:
            flag=1
        target_seq = np.zeros((1, 1, len(french_character)))
        target_seq[0, 0,sample] = 1.0
        states_value = [h, c,h1,c1]
    return sent

In [37]:
english_texts[5]

'Who?'

In [38]:
french_texts[5]

'\tQui ?\n'

In [39]:
english='Who?'

In [40]:
k=len(english)
m=0
a=[]
b=[]
c=[]
inpu=[]
while m<k:
    for char in english[m]:
        for i in range(len(english_character)):
            if english_d[char]==i:
                a.append(1)
            else:
                a.append(0)
    for kp in a:
        b.append(kp)
    c.append(b)
    b=[]
    a=[]
    m=m+1
while m<max_encoder_seq_length:
        for i in range(len(english_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
inpu.append(c)

In [41]:
inpu=np.array(inpu)

In [42]:
inpu

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]]])

In [43]:
inpu.shape

(1, 20, 75)

In [44]:
d=decode_sequence(inpu)

In [45]:
print(d)

Qui ?

